In [3]:
from cffi import model
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans

In [2]:
spark = SparkSession.builder.appName("Clustering").getOrCreate()

24/01/13 23:07:57 WARN Utils: Your hostname, DESKTOP-0HJMRJF resolves to a loopback address: 127.0.1.1; using 172.20.8.86 instead (on interface eth0)
24/01/13 23:07:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/13 23:07:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
data = spark.read.format("libsvm").load('./data/ML/clustering/sample_kmeans_data.txt')

24/01/13 23:11:02 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [6]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|           (3,[],[])|
|  1.0|(3,[0,1,2],[0.1,0...|
|  2.0|(3,[0,1,2],[0.2,0...|
|  3.0|(3,[0,1,2],[9.0,9...|
|  4.0|(3,[0,1,2],[9.1,9...|
|  5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+


In [7]:
data.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)


In [8]:
modify_data = data.select(['features'])
modify_data.show()

+--------------------+
|            features|
+--------------------+
|           (3,[],[])|
|(3,[0,1,2],[0.1,0...|
|(3,[0,1,2],[0.2,0...|
|(3,[0,1,2],[9.0,9...|
|(3,[0,1,2],[9.1,9...|
|(3,[0,1,2],[9.2,9...|
+--------------------+


In [12]:
kmeans = KMeans(featuresCol='features').setK(2).setSeed(1)

In [19]:
kmeans_model = kmeans.fit(modify_data)

In [17]:
kmeans_model

KMeansModel: uid=KMeans_771bcd629fcd, k=2, distanceMeasure=euclidean, numFeatures=3

In [25]:
wssse= kmeans_model.summary.trainingCost
wssse

0.11999999999994547

In [26]:
centers = kmeans_model.clusterCenters()
centers

[array([9.1, 9.1, 9.1]), array([0.1, 0.1, 0.1])]

In [28]:
results = kmeans_model.transform(modify_data).show()
results

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         1|
|(3,[0,1,2],[0.1,0...|         1|
|(3,[0,1,2],[0.2,0...|         1|
|(3,[0,1,2],[9.0,9...|         0|
|(3,[0,1,2],[9.1,9...|         0|
|(3,[0,1,2],[9.2,9...|         0|
+--------------------+----------+
